In [ ]:
import duckdb
import pandas as pd
from google.cloud import storage
from google.colab import auth
auth.authenticate_user()


In [ ]:
goss_1_bjas2 = pd.read_csv('gs://buk-tlsssr-dmeyf/exp/entrega3_goss_1_bjas2/entrega3_goss_1_bjas2_predicciones.csv')
goss_1_bjas1 = pd.read_csv('gs://buk-tlsssr-dmeyf/exp/entrega3_goss_1_bjas1/entrega3_goss_1_bjas1_predicciones.csv')
goss_2_bjas2 = pd.read_csv('gs://buk-tlsssr-dmeyf/exp/entrega3_goss_2_bjas2/entrega3_goss_2_bjas2_predicciones.csv')
sinLags_1_bjas1 = pd.read_csv('gs://buk-tlsssr-dmeyf/exp/entrega3_sinLags_1_bjas1/entrega3_sinLags_1_bjas1_predicciones.csv')
sinLags_1_bjas2 = pd.read_csv('gs://buk-tlsssr-dmeyf/exp/entrega3_sinLags_1_bjas2/entrega3_sinLags_1_bjas2_predicciones.csv')

In [ ]:
datasets_list = [goss_1_bjas2, sinLags_1_bjas1, sinLags_1_bjas2, goss_1_bjas1,  goss_2_bjas2]
names = ['goss_1_bjas2', 'sinLags_1_bjas1', 'sinLags_1_bjas2', 'goss_1_bjas1',  'goss_2_bjas2']

result_df = datasets_list[0][['numero_de_cliente', 'foto_mes']].copy()

for i, df in enumerate(datasets_list):
    result_df = pd.merge(result_df, df[['numero_de_cliente', 'foto_mes', 'pred_11']], on=['numero_de_cliente', 'foto_mes'])
    result_df.rename(columns={'pred_11': names[i]}, inplace=True)

result_df['p_final'] =( 0.0667 * result_df['sinLags_1_bjas1'] + 0.2444 * result_df['sinLags_1_bjas2']
                         +  0.2444 * result_df['goss_1_bjas1']) + 0.3333 * result_df['goss_1_bjas2'] + 0.1111 *result_df['goss_2_bjas2']

In [ ]:
import numpy as np

entrega = result_df[['numero_de_cliente', 'p_final']].sort_values(by='p_final', ascending=False).reset_index(drop=True)
entrega['Predicted'] = np.where(entrega.index < 11500, 1, 0)
entrega = entrega[['numero_de_cliente', 'Predicted']]
entrega.to_csv('entrega_goss1_bjas2_12500.csv', index=False)